In [16]:
from MulticoreTSNE import MulticoreTSNE as TSNE
from matplotlib import pyplot as plt
from tools import utils
import torch

scaler = torch.cuda.amp.GradScaler()

In [9]:
pretrained = 'weights/supcon_first_stage/epoch35'
data_dir = 'data/product_version3'
batch_sizes = {
    "train_batch_size": 128,
    'valid_batch_size': 128
}
num_workers = 12
backbone = 'resnet18'

transforms = utils.build_transforms(False)
loaders = utils.build_loaders(data_dir, transforms, batch_sizes, num_workers)

In [15]:
model = utils.build_model(backbone).cuda()
model.load_state_dict(torch.load(pretrained))
model.eval()

embeddings, labels = utils.compute_embeddings(loaders['valid_loader'], model, scaler)
embeddings_tsne = TSNE(n_jobs=12).fit_transform(embeddings)
vis_x = embeddings_tsne[:, 0]
vis_y = embeddings_tsne[:, 1]
plt.scatter(vis_x, vis_y, c=labels, cmap=plt.cm.get_cmap("jet", 44), marker='.')
plt.colorbar(ticks=range(44))
plt.show()

NameError: name 'F' is not defined